## Preparation

In [158]:
import os
import requests
import json
import pandas as pd # type: ignore
import time
import itertools
# import datetime
from dotenv import load_dotenv # type: ignore

In [159]:
# read env
load_dotenv()
apikey = os.getenv("APIKey")

In [160]:
# uncomment if used in Google Colab. Setup the API key in Google Colab (see README.md)
# from google.colab import userdata
# apikey = userdata.get('apiKey')

## Using Scopus API

Define keywords and ISSN

In [161]:
# See list of ISSN in 'Database of Sources.csv'
issn = '1743-2774' # Cartographic Journal
# issn = '1365-8816' # IJGIS
# issn = '1467-9671' # Transactions in GIS
# issn = '1077-2626' # TVCG
# issn = '2165-8773' # PacivicVIS
# issn = '1062-9432' # ACM CHI
# issn = '1753-8955' # IJDE
# issn = '1744-5647' # Journal of Maps
# issn = '1946-7729' # ACM SIGSPATIAL
# issn = '1948-660X' #JOSIS
# issn = '2570-2084' # The Advances in Cartography and GIScience
# issn = '2220-9964' # IJGI
# issn = '1523-0406' # Cartography and Geographic Information Science
# issn = '1048-9053' # Cartographic Perspective
# issn = '2372-9333' # International Journal of Cartography
issn = '0317-7173' # Cartographica


In [162]:
keywords = ['carto*', 'visual*', 'geovis*'] # multivariate OR bivariate are always included 
# keywords = ['bivariate', 'multivariate', 'visual*', 'carto*', 'geovis*']
# keywords = ['bivariate', 'multivariate', 'visual*'] # for testing

## Permutations of Search Queries

In [163]:
def generateKeywords(keywords, issn):
  """
  Create a list of search queries from permutations and combinations of keywords
  """

  searchQuery = []

  # Generate all combinations of 2 first keywords from the list
  combinations = list(itertools.combinations(keywords, 2))

  # For each combination, generate permutations with the remaining keywords
  for combo in combinations:
      remaining_keywords = [k for k in keywords if k not in combo]
      for keyword in remaining_keywords:
          # query = f"({combo[0]} AND {combo[1]} OR {keyword})"
          query = f"(TITLE-ABS-KEY((multivariate OR bivariate) AND (({combo[0]} AND {combo[1]}) OR {keyword})) AND ISSN({issn})) AND (PUBYEAR AFT 2009 AND PUBYEAR BEF 2024)"
          searchQuery.append(query)

  return searchQuery

In [164]:
queryList = generateKeywords(keywords, issn)

In [165]:
queryList

['(TITLE-ABS-KEY((multivariate OR bivariate) AND ((carto* AND visual*) OR geovis*)) AND ISSN(0317-7173)) AND (PUBYEAR AFT 2009 AND PUBYEAR BEF 2024)',
 '(TITLE-ABS-KEY((multivariate OR bivariate) AND ((carto* AND geovis*) OR visual*)) AND ISSN(0317-7173)) AND (PUBYEAR AFT 2009 AND PUBYEAR BEF 2024)',
 '(TITLE-ABS-KEY((multivariate OR bivariate) AND ((visual* AND geovis*) OR carto*)) AND ISSN(0317-7173)) AND (PUBYEAR AFT 2009 AND PUBYEAR BEF 2024)']

## Get the papers

In [166]:
def getPapers(queryList, issn):
    """
    get papers from Scopus API for all queries in the list
    """

    outputFolder = os.path.join(os.getcwd(), 'output', issn)

    # Check if the output dir exists
    if not os.path.exists(outputFolder):
        os.makedirs(outputFolder)

    for query in queryList:
        # for every query, search and fetch all results from every page
        print(f"\nWorking on Query: {query}")
        start_index = 0 # start index for pagination
        all_papers = [] # store all papers from every page
        all_data = [] # store all raw data from every page
        batch = 1
        index = queryList.index(query)+1

        # repeat until all results are fetched from every page
        while True:
            print(f"   Running batch: {batch} from query {index}")
            url = "https://api.elsevier.com/content/search/scopus"
            headers = {
                "Accept": "application/json",
                "X-ELS-APIKey": apikey
            }
            params = {
                "query": query,
                "sort": "date",
                "start": start_index
            }

            response = requests.get(url, headers=headers, params=params)
            # print(f'Response headers.. {response.headers}') # for debugging rate limits
            
            if response.status_code == 200:
                data = json.loads(response.text)
                all_data.append(data)

                # Extract specified attributes
                entries = data['search-results']['entry']
                papers = []
                for entry in entries:
                    paper = {
                        "doi": entry.get("prism:doi"),
                        "title": entry.get("dc:title"),
                        "authors": entry.get("dc:creator"),
                        "year": entry.get("prism:coverDisplayDate"),
                        "publicationName": entry.get("prism:publicationName"),
                        "url": entry.get("prism:url"),
                        "citedby-count": entry.get("citedby-count")
                    }
                    papers.append(paper)

                # combine all data from paginations
                all_papers.extend(papers)
                total_results = int(data['search-results']['opensearch:totalResults'])
                start_index += 25  # Increase the start index for the next page of results
                print(f"Search result for this iteration: page {start_index} from {total_results}")
                
                # Save all raw data of this search to JSON file
                raw_filename = 'raw_'+issn+'_index'+str(index)+'_batch'+str(batch)+'.json'
                with open(os.path.join(outputFolder, raw_filename), 'w') as f:
                    json.dump(all_data, f)
                if start_index > total_results:
                    break
                time.sleep(60)  # Pause for 60 seconds to deal with rate limits
            else:
                print("Failed to fetch papers.")
                break
            batch += 1

        # Load all papers from the single search into pandas DataFrame
        df = pd.DataFrame(all_papers)
        
        # Specify the file name of this search
        csv_filename = 'papers_'+issn+'_index'+str(index)+'.csv'
        csv_file_path = os.path.join(outputFolder, csv_filename)
        df.to_csv(csv_file_path)
                
    # return final_df
    dfs = []
    # Loop over all files in the directory
    for filename in os.listdir(outputFolder):
        # Check if the file is a CSV file
        if filename.endswith('.csv'):
            print(f"Reading file: {filename}")
            filepath = os.path.join(outputFolder, filename)
            df_file = pd.read_csv(filepath)
            dfs.append(df_file)

    combined_df = pd.concat(dfs, ignore_index=True)
    final_df = combined_df.drop_duplicates(subset='doi')
    cleaned_csv_filename = 'cleanedPapers_'+issn+'.csv'
    final_df.to_csv(os.path.join(outputFolder, cleaned_csv_filename), index=False)
    print(f"Total papers found: {len(final_df)}")

    return final_df

In [167]:
final_dataframe = getPapers(queryList, issn)


Working on Query: (TITLE-ABS-KEY((multivariate OR bivariate) AND ((carto* AND visual*) OR geovis*)) AND ISSN(0317-7173)) AND (PUBYEAR AFT 2009 AND PUBYEAR BEF 2024)
   Running batch: 1 from query 1


Search result for this iteration: page 25 from 1

Working on Query: (TITLE-ABS-KEY((multivariate OR bivariate) AND ((carto* AND geovis*) OR visual*)) AND ISSN(0317-7173)) AND (PUBYEAR AFT 2009 AND PUBYEAR BEF 2024)
   Running batch: 1 from query 2
Search result for this iteration: page 25 from 1

Working on Query: (TITLE-ABS-KEY((multivariate OR bivariate) AND ((visual* AND geovis*) OR carto*)) AND ISSN(0317-7173)) AND (PUBYEAR AFT 2009 AND PUBYEAR BEF 2024)
   Running batch: 1 from query 3
Search result for this iteration: page 25 from 1
Reading file: papers_0317-7173_index2.csv
Reading file: papers_0317-7173_index1.csv
Reading file: cleanedPapers_0317-7173.csv
Reading file: papers_0317-7173_index3.csv
Total papers found: 1


---